In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import Counter
import tensorflow.keras as keras
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import keras_cv
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import requests
from tensorflow.keras.layers import Dense, Input, Flatten, Conv2D, BatchNormalization, Activation, Dropout, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
from IPython.display import YouTubeVideo

C:\Users\Kasparas\anaconda3\envs\tf\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\Kasparas\anaconda3\envs\tf\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = np.load('flatland_train.npz')
X = data['X']
y = data['y']

y[y != 0] -= 2    # Correct labels so that triangle is mapped to class 1
X = X / 255.      # Scale down to range [0, 1]

MemoryError: Unable to allocate 191. MiB for an array with shape (25000000,) and data type float64

In [ ]:
X_turbo=np.concatenate((X,X,X,X,X,X))
y_turbo=np.concatenate((y,y,y,y,y,y))
X_turbo = np.expand_dims(X_turbo, axis=3)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=45,
    width_shift_range=0.55,
    height_shift_range=0.55,
    zoom_range = [0.35, 1.65],
    shear_range = 13,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest')

In [ ]:
initial_learning_rate = 4e-4
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.98,
    staircase=True)

In [46]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(16, kernel_size=(9, 9), activation='relu', input_shape=[50, 50, 1]))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(keras.layers.Dense(10, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              metrics=["accuracy"])
model.summary()

loss = model.fit(datagen.flow(X_turbo,y_turbo,batch_size=32), epochs=150)

Model: "sequential_16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_84 (Conv2D)          (None, 42, 42, 16)        1312      
                                                                 
 conv2d_85 (Conv2D)          (None, 40, 40, 16)        2320      
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 20, 20, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_86 (Conv2D)          (None, 18, 18, 32)        4640      
                                                                 
 conv2d_87 (Conv2D)          (None, 16, 16, 32)        9248      
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 8, 8, 32)         0         
 g2D)                                                

1875/1875 [==============================] - 23s 12ms/step - loss: 0.4713 - accuracy: 0.8209
Epoch 136/150
1875/1875 [==============================] - 24s 13ms/step - loss: 0.4607 - accuracy: 0.8253
Epoch 137/150
1875/1875 [==============================] - 23s 12ms/step - loss: 0.4719 - accuracy: 0.8217
Epoch 138/150
1875/1875 [==============================] - 24s 13ms/step - loss: 0.4650 - accuracy: 0.8261
Epoch 139/150
1875/1875 [==============================] - 23s 12ms/step - loss: 0.4657 - accuracy: 0.8235
Epoch 140/150
1875/1875 [==============================] - 23s 12ms/step - loss: 0.4643 - accuracy: 0.8238
Epoch 141/150
1875/1875 [==============================] - 23s 12ms/step - loss: 0.4689 - accuracy: 0.8235
Epoch 142/150
1875/1875 [==============================] - 24s 13ms/step - loss: 0.4671 - accuracy: 0.8229
Epoch 143/150
1875/1875 [==============================] - 23s 12ms/step - loss: 0.4678 - accuracy: 0.8221
Epoch 144/150
1875/1875 [==========================

In [47]:
model.save('model.h5')
#Train set accuracy 98.52% Test set (simple) accuracy 100.00% Test set (adv.) accuracy 100.00%

In [29]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size=(9, 9), activation='relu', input_shape=[50, 50, 1]))

model.add(keras.layers.Conv2D(8, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(keras.layers.Dense(5, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              metrics=["accuracy"])
model.summary()

loss = model.fit(datagen.flow(X_turbo,y_turbo,batch_size=128), epochs=220)

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_95 (Conv2D)          (None, 42, 42, 8)         656       
                                                                 
 conv2d_96 (Conv2D)          (None, 40, 40, 8)         584       
                                                                 
 max_pooling2d_45 (MaxPoolin  (None, 20, 20, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_97 (Conv2D)          (None, 18, 18, 16)        1168      
                                                                 
 conv2d_98 (Conv2D)          (None, 16, 16, 16)        2320      
                                                                 
 max_pooling2d_46 (MaxPoolin  (None, 8, 8, 16)         0         
 g2D)                                                

ResourceExhaustedError: Graph execution error:

2 root error(s) found.
  (0) RESOURCE_EXHAUSTED:  MemoryError: Unable to allocate 1.22 MiB for an array with shape (128, 50, 50, 1) and data type float32
Traceback (most recent call last):

  File "C:\Users\Kasparas\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "C:\Users\Kasparas\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "C:\Users\Kasparas\AppData\Roaming\Python\Python39\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "C:\Users\Kasparas\anaconda3\envs\tf\lib\site-packages\keras\engine\data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  File "C:\Users\Kasparas\anaconda3\envs\tf\lib\site-packages\keras\eng

In [ ]:
model.save('Featherweight_model.h5')
#Train set accuracy 98.51% Test set (simple) accuracy 99.98% Test set (adv.) accuracy 99.98%
#2x8 -> 2x16 -> 3x32 150 epochs 16 batch size

In [26]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size=(9, 9), activation='relu', input_shape=[50, 50, 1]))

model.add(keras.layers.Conv2D(8, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(keras.layers.Dense(5, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              metrics=["accuracy"])
model.summary()

loss = model.fit(datagen.flow(X_turbo,y_turbo,batch_size=32), epochs=220)

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_83 (Conv2D)          (None, 42, 42, 8)         656       
                                                                 
 conv2d_84 (Conv2D)          (None, 40, 40, 8)         584       
                                                                 
 max_pooling2d_39 (MaxPoolin  (None, 20, 20, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_85 (Conv2D)          (None, 18, 18, 16)        1168      
                                                                 
 conv2d_86 (Conv2D)          (None, 16, 16, 16)        2320      
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 8, 8, 16)         0         
 g2D)                                                

1875/1875 [==============================] - 22s 12ms/step - loss: 0.5580 - accuracy: 0.7863
Epoch 136/220
1875/1875 [==============================] - 22s 12ms/step - loss: 0.5552 - accuracy: 0.7854
Epoch 137/220
1875/1875 [==============================] - 23s 12ms/step - loss: 0.5621 - accuracy: 0.7836
Epoch 138/220
1875/1875 [==============================] - 23s 12ms/step - loss: 0.5605 - accuracy: 0.7833
Epoch 139/220
1875/1875 [==============================] - 22s 12ms/step - loss: 0.5547 - accuracy: 0.7857
Epoch 140/220
1875/1875 [==============================] - 23s 12ms/step - loss: 0.5558 - accuracy: 0.7849
Epoch 141/220
1875/1875 [==============================] - 23s 12ms/step - loss: 0.5561 - accuracy: 0.7879
Epoch 142/220
1875/1875 [==============================] - 23s 12ms/step - loss: 0.5564 - accuracy: 0.7855
Epoch 143/220
1875/1875 [==============================] - 24s 13ms/step - loss: 0.5577 - accuracy: 0.7862
Epoch 144/220
1875/1875 [==========================

In [27]:
model.save('Featherweight_model2.h5')

In [28]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(8, kernel_size=(11, 11), activation='relu', input_shape=[50, 50, 1]))

model.add(keras.layers.Conv2D(8, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu'))

model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(keras.layers.Dense(5, activation="softmax"))
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              metrics=["accuracy"])
model.summary()

loss = model.fit(datagen.flow(X_turbo,y_turbo,batch_size=32), epochs=220)

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_89 (Conv2D)          (None, 40, 40, 8)         976       
                                                                 
 conv2d_90 (Conv2D)          (None, 38, 38, 8)         584       
                                                                 
 max_pooling2d_42 (MaxPoolin  (None, 19, 19, 8)        0         
 g2D)                                                            
                                                                 
 conv2d_91 (Conv2D)          (None, 17, 17, 16)        1168      
                                                                 
 conv2d_92 (Conv2D)          (None, 15, 15, 16)        2320      
                                                                 
 max_pooling2d_43 (MaxPoolin  (None, 7, 7, 16)         0         
 g2D)                                                

MemoryError: Unable to allocate 572. MiB for an array with shape (60000, 50, 50, 1) and data type float32

In [ ]:
model.save('Featherweight_model3.h5')